# Filtrage des ondes équatoriales dans les données quotidiennes d'OLR

**Auteur : FERRY Frédéric (ENM/C3M) - Novembre 2021**

Filtrage en nombre d'onde et en fréquence de données quotidiennes d'OLR afin d'extraire les signaux des ondes équatoriales et de la MJO.

D'après https://ncics.org/portfolio/monitor/mjo/

In [ ]:
%matplotlib inline

import os
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import scipy
import math
from scipy import signal,fftpack, interpolate
from scipy.interpolate import griddata

import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid
from cartopy.util import add_cyclic_point

import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

mpl.rcParams['mathtext.default'] = 'regular'

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

import warnings
warnings.filterwarnings("ignore")

In [ ]:
dir_data='./data/'
dir_figs='./figs/'
dir_anim='./anim/'
if not os.path.exists(dir_figs):
    os.makedirs(dir_figs)
if not os.path.exists(dir_anim):
    os.makedirs(dir_anim)

# Calcul des anomalies quotidiennes

In [ ]:
year1='1980'
year2='2020'
latS=-15.
latN=15.

olr    = xr.open_dataset(dir_data+'olr.day.mean.nc').sel(time=slice(year1,year2)).sel(lat=slice(latN,latS))

print(' ----- Computing daily anomalies ----- ')
olr_anom=olr.groupby('time.dayofyear') - olr.groupby('time.dayofyear').mean('time')

print(olr_anom)

# Filtrage spectral

In [ ]:
def kf_filter(inData,obsPerDay,tMin,tMax,kMin,kMax,hMin,hMax,waveName):
	timeDim = inData.shape[0]
	lonDim = inData.shape[1]
	
	# Reshape data from [time,lon] to [lon,time]
	originalData=np.zeros([lonDim,timeDim],dtype='f')
	for counterX in range(timeDim):
	    test=0
	    for counterY in range(lonDim-1,-1,-1):
	        originalData[test,counterX]=inData[counterX,counterY]
	        test+=1
	
	# Detrend the Data
	detrendData=np.zeros([lonDim,timeDim],dtype='f')
	for counterX in range(lonDim):
	    detrendData[counterX,:]=signal.detrend(originalData[counterX,:])
	
	# Taper 
	taper=signal.tukey(timeDim,0.05,True)
	taperData=np.zeros([lonDim,timeDim],dtype='f')
	for counterX in range(lonDim):
	    taperData[counterX,:]=detrendData[counterX,:]*taper
	
	# Perform 2-D Fourier Transform
	fftData=np.fft.rfft2(taperData)
	kDim=lonDim 
	freqDim=round(fftData.shape[1])
	
	# Find the indeces for the period cut-offs
	jMin = int(round( ( timeDim * 1. / ( tMax * obsPerDay ) ), 0 ))
	jMax = int(round( ( timeDim * 1. / ( tMin * obsPerDay ) ), 0 ))
	jMax = min( ( jMax, freqDim ) )

	# Find the indices for the wavenumber cut-offs
	# This is more complicated because east and west are separate
	if( kMin < 0 ):
	    iMin = round( ( kDim + kMin ), 3 )
	    iMin = max( ( iMin, ( kDim / 2 ) ) )
	else:
	    iMin = round( kMin, 3 )
	    iMin = min( ( iMin, ( kDim / 2 ) ) )

	if( kMax < 0 ):
	    iMax = round( ( kDim + kMax ), 3 )
	    iMax = max( ( iMax, ( kDim / 2 ) ) )
	else:
	    iMax = round( kMax, 3 )
	    iMax = min( ( iMax, ( kDim / 2 ) ) )
	  
	# set the appropriate coefficients to zero
	iMin=int(iMin)
	iMax=int(iMax)
	jMin=int(jMin)
	jMax=int(jMax)
	if( jMin > 0 ):
	    fftData[:, :jMin-1 ] = 0
	if( jMax < ( freqDim - 1 ) ):
	    fftData[:, jMax+1: ] = 0

	if( iMin < iMax ):
	    # Set things outside the range to zero, this is more normal
	    if( iMin > 0 ):
	        fftData[:iMin-1, : ] = 0
	    if( iMax < ( kDim - 1 ) ):
	        fftData[iMax+1:, : ] = 0
	else:
	    # Set things inside the range to zero, this should be somewhat unusual
	    fftData[iMax+1:iMin-1, : ] = 0
	
	# Find constants
	PI = math.pi
	beta = 2.28e-11
	if hMin != -999:
	    cMin = float( 9.8 * float(hMin) )**0.5
	else:
	    cMin=hMin
	if hMax != -999:
	    cMax = float( 9.8 * float(hMax) )**0.5
	else:
	    cMax=hMax
	c = np.array([cMin,cMax])
	spc = 24 * 3600. / ( 2 * PI * obsPerDay ) # seconds per cycle
	
	# Now set things to zero that are outside the Kelvin dispersion
	for i in range(0,kDim):
		# Find Non-Dimensional WaveNumber (k)
		if( i > ( kDim / 2 ) ):
			# k is negative
			k = ( i - kDim  ) * 1. / (6.37e6) # adjusting for circumfrence of earth
		else:
			# k is positive
			k = i * 1. / (6.37e6) # adjusting for circumfrence of earth
		
		# Find Frequency
		freq = np.array([ 0, freqDim * (1. / spc) ]) #waveName='None'
		jMinWave = 0
		jMaxWave = freqDim	
		if waveName.lower() == "kelvin":
		    freq = k * c
		if waveName.lower() == "er":
		    freq = -beta * k / ( k**2 + 3. * beta / c )
		if waveName.lower() == "ig1":
		    freq = ( 3 * beta * c + k**2 * c**2 )**0.5
		if waveName.lower() == "ig2":
		    freq = ( 5 * beta * c + k**2 * c**2 )**0.5
		if waveName.lower() == "mrg" or waveName.lower()=="ig0":   	
			if( k == 0 ):
				freq = ( beta * c )**0.5
			else:
				if( k > 0):
					freq = k * c * ( 0.5 + 0.5 * ( 1 + 4 * beta / ( k**2 * c ) )**0.5 )
				else:
					freq = k * c * ( 0.5 - 0.5 * ( 1 + 4 * beta / ( k**2 * c ) )**0.5 )	
		
		# Get Min/Max Wave 
		if(hMin==mis):
		    jMinWave = 0
		else:
		    jMinWave = int( math.floor( freq[0] * spc * timeDim ) )

		if(hMax==mis):
		    jMaxWave = freqDim
		else:
		    jMaxWave = int( math.ceil( freq[1] * spc * timeDim ) )

		jMaxWave = max(jMaxWave, 0)
		jMinWave = min(jMinWave, freqDim)
		
		# set the appropriate coefficients to zero
		i=int(i)
		jMinWave=int(jMinWave)
		jMaxWave=int(jMaxWave)
		if( jMinWave > 0 ):
		    fftData[i, :jMinWave-1] = 0
		if( jMaxWave < ( freqDim - 1 ) ):
		    fftData[i, jMaxWave+1:] = 0
	
	# perform the inverse transform to reconstruct the data
	returnedData=np.fft.irfft2(fftData) 
	
	# Reshape data from [lon,time] to [time,lon]
	outData=np.zeros([timeDim,lonDim],dtype='f')
	for counterX in range(returnedData.shape[1]):
	    test=0
	    for counterY in range(lonDim-1,-1,-1):
	        outData[counterX,counterY]=returnedData[test,counterX] 
	        test+=1
    
    # Return Result
	return outData

In [ ]:
date1=input('Entrer la date de début au format YYYY-MM-DD : ')

In [ ]:
# Données entre 15S et 15N
data_olr=olr_anom.sel(lat=slice(15,-15))
data=data_olr['olr']
lat = data_olr.lat.values
lon = data_olr.lon.values
time = data_olr.time.values

# Période totale des données
print(time)
d1=time[0]
d2=time[-1]
dates = pd.date_range(d1, d2, freq='D')
print(dates)

# Choix des données sur une période de 3 mois
startDate=dates.get_loc(date1)
endDate=startDate+91
anom=data[startDate:endDate,:,:]
time_hov=data[startDate:endDate,:,:].time.values

time_str=[x for x in range(len(time_hov))]
date_str=[x for x in range(len(time_hov))]

for i in range(len(time_hov)):
	time_str[i] = str(time_hov[i])
	date_str[i] = time_str[i][0:10]

print(date_str)

# Ajout de 90j autour de la période d'étude pour le filtrage
bufferDays = 90
filtStart = startDate - bufferDays
filtEnd   = endDate + bufferDays
filtData=data[filtStart:filtEnd,:,:]
filtTime=filtData.shape[0]

In [ ]:
backup=np.array(filtData,dtype='f')

<img src="spectra.png" alt="drawing" width="400"/>

In [ ]:
# Filter MJO : Kiladis et al. (2005 JAS) for 20-100
mjo_wavenumber=[0,9]
mjo_period=[20,100]

# Filter Kelvin Straub & Kiladis (2002) to 20 days
kelvin_wavenumber=[1,14]
kelvin_period=[2.5,20]

# Filter ER : Kiladis et al. (2009 Rev. Geophys.)
er_wavenumber=[-10,-1]
er_period=[9.7,72]

# Filter MRG : Wheeler & Kiladis (1999)
mrg_wavenumber=[-10,-1]
mrg_period=[3,96]

# Filter (Low) >120d
low_wavenumber=[-10,10]
low_period=[120,9999]

In [ ]:
mis = -999
obsPerDay = 1 # données quotidiennes

#################################################
# Filter (MJO)
#################################################
print("\n##############################\nFiltering (MJO)\n")

mjo = np.array(filtData,dtype='f')
mjo_wavenumber=np.array(mjo_wavenumber,dtype='f')
mjo_period=np.array(mjo_period,dtype='f')
mjo_depth=np.array([mis,mis],dtype='f')

for lat_counter in range(0,len(lat)):
	print("###########################################################################################")
	print(lat_counter,lat[lat_counter])
	mjo[:,lat_counter,:]=kf_filter(filtData[:,lat_counter,:],obsPerDay,\
		mjo_period[0],mjo_period[1],\
		mjo_wavenumber[0],mjo_wavenumber[1],\
		mjo_depth[0],mjo_depth[1],"none")
	print(np.min(mjo[:,lat_counter,:]),np.max(mjo[:,lat_counter,:]))

# Reset FiltData
filtData=np.zeros([filtTime,len(lat),len(lon)],dtype='f')
for x in range(0,filtData.shape[0]):
  for y in range(0,filtData.shape[1]):
    for z in range(0,filtData.shape[2]):
      filtData[x,y,z]=backup[x,y,z]

#################################################
# Filter (ER)
#################################################
print("\n##############################\nFiltering (ER)\n")

er = filtData
er_wavenumber=np.array(er_wavenumber,dtype='f')
er_period=np.array(er_period,dtype='f')
er_depth=np.array([mis,mis],dtype='f')

for lat_counter in range(0,len(lat)):
	print("###########################################################################################")
	print(lat_counter,lat[lat_counter])
	er[:,lat_counter,:]=kf_filter(filtData[:,lat_counter,:],obsPerDay,\
		er_period[0],er_period[1],\
		er_wavenumber[0],er_wavenumber[1],\
		er_depth[0],er_depth[1],"er")
	print(np.min(er[:,lat_counter,:]),np.max(er[:,lat_counter,:]))

# Reset FiltData
filtData=np.zeros([filtTime,len(lat),len(lon)],dtype='f')
for x in range(0,filtData.shape[0]):
  for y in range(0,filtData.shape[1]):
    for z in range(0,filtData.shape[2]):
      filtData[x,y,z]=backup[x,y,z]

#################################################
# Filter (Low)
#################################################
print("\n##############################\nFiltering (Low)\n")

low = filtData
low_wavenumber=np.array(low_wavenumber,dtype='f')
low_period=np.array(low_period,dtype='f')
low_depth=np.array([mis,mis],dtype='f')

for lat_counter in range(0,len(lat)):
	print("###########################################################################################")
	print(lat_counter,lat[lat_counter])
	low[:,lat_counter,:]=kf_filter(filtData[:,lat_counter,:],obsPerDay,\
		low_period[0],low_period[1],\
		low_wavenumber[0],low_wavenumber[1],\
		low_depth[0],low_depth[1],"none")
	print(np.min(low[:,lat_counter,:]),np.max(low[:,lat_counter,:]))

# Reset FiltData
filtData=np.zeros([filtTime,len(lat),len(lon)],dtype='f')
for x in range(0,filtData.shape[0]):
  for y in range(0,filtData.shape[1]):
    for z in range(0,filtData.shape[2]):
      filtData[x,y,z]=backup[x,y,z]

#################################################
# Filter (KELVIN)
#################################################
print("\n##############################\nFiltering (KELVIN)\n")

kelvin = filtData
kelvin_wavenumber=np.array(kelvin_wavenumber,dtype='f')
kelvin_period=np.array(kelvin_period,dtype='f')
kelvin_depth=np.array([8,90],dtype='f')

for lat_counter in range(0,len(lat)):
	print("###########################################################################################")
	print(lat_counter,lat[lat_counter])
	
	kelvin[:,lat_counter,:]=kf_filter(filtData[:,lat_counter,:],obsPerDay,\
	           kelvin_period[0],kelvin_period[1],\
	           kelvin_wavenumber[0],kelvin_wavenumber[1],\
	           kelvin_depth[0],kelvin_depth[1],"kelvin")
	print(np.min(kelvin[:,lat_counter,:]),np.max(kelvin[:,lat_counter,:]))

# Reset FiltData
filtData=np.zeros([filtTime,len(lat),len(lon)],dtype='f')
for x in range(0,filtData.shape[0]):
  for y in range(0,filtData.shape[1]):
    for z in range(0,filtData.shape[2]):
      filtData[x,y,z]=backup[x,y,z]
    
#################################################
# Filter (MRG)
#################################################
print("\n##############################\nFiltering (MRG)\n")

mrg = filtData
mrg_wavenumber=np.array(mrg_wavenumber,dtype='f')
mrg_period=np.array(mrg_period,dtype='f')
mrg_depth=np.array([8,90],dtype='f')

for lat_counter in range(0,len(lat)):
	print("###########################################################################################")
	print(lat_counter,lat[lat_counter])
	
	mrg[:,lat_counter,:]=kf_filter(filtData[:,lat_counter,:],obsPerDay,\
	           mrg_period[0],mrg_period[1],\
	           mrg_wavenumber[0],mrg_wavenumber[1],\
	           mrg_depth[0],mrg_depth[1],"mrg")
	print(np.min(mrg[:,lat_counter,:]),np.max(mrg[:,lat_counter,:]))

# Reset FiltData
filtData=np.zeros([filtTime,len(lat),len(lon)],dtype='f')
for x in range(0,filtData.shape[0]):
  for y in range(0,filtData.shape[1]):
    for z in range(0,filtData.shape[2]):
      filtData[x,y,z]=backup[x,y,z]

# Tracé des champs

In [ ]:
mjo2=mjo[bufferDays:-bufferDays,:,:]
er2=er[bufferDays:-bufferDays,:,:]
kelvin2=kelvin[bufferDays:-bufferDays,:,:]
mrg2=mrg[bufferDays:-bufferDays,:,:]
low2=low[bufferDays:-bufferDays,:,:]

In [ ]:
data_hov=backup[bufferDays:-bufferDays,:,:].mean(axis=1)
mjo_hov=mjo2.mean(axis=1)
er_hov=er2.mean(axis=1)
kelvin_hov=kelvin2.mean(axis=1)
mrg_hov=mrg2.mean(axis=1)
low_hov=low2.mean(axis=1)

print(data_hov.shape)
print(mjo_hov.shape)

In [ ]:
levels_olr = np.arange(-160,180,20)
levels_mjo_neg = np.arange(-50,-5,5)
levels_mjo_pos = np.arange(0,55,5)
levels_kelvin = np.arange(-50,-5,5)
levels_er = np.arange(-50,-5,5)
levels_mrg = np.arange(-50,-5,5)
levels_low = np.arange(-20,-2,2)

lonW=lon[0]
lonE=lon[-1]

bounds = [(lonW, lonE, -15, 15)]

def plot_background(ax):
   ax.set_xticks(np.linspace(0, 360, 13), crs=ccrs.PlateCarree())
   ax.set_yticks(np.linspace(latS, latN, 5), crs=ccrs.PlateCarree())
   ax.axes.axis('tight')
   lon_formatter = LongitudeFormatter(zero_direction_label=True)
   lat_formatter = LatitudeFormatter()
   ax.xaxis.set_major_formatter(lon_formatter)
   ax.yaxis.set_major_formatter(lat_formatter)
   ax.coastlines()
   ax.set_extent(*bounds, crs=ccrs.PlateCarree())
   return ax

for i in range(len(time_hov)):
    print(date_str[i])
    fig = plt.figure(figsize=(15., 17.))
    fig.suptitle('Wave filtering in OLR anomalies : '+date_str[i], fontsize=16)

    ax = fig.add_subplot(511, projection=ccrs.PlateCarree(central_longitude=180.0))
    ax.set_title('OLR anomalies and Low : '+date_str[i], loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, backup[90+i,:,:], levels_olr,  transform=ccrs.PlateCarree(), cmap='RdBu_r', extend='both')
    c = ax.contour(lon, lat, low2[i,:,:], levels_low[levels_low != 0], colors='purple', linestyles="-", transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
    
    ax = fig.add_subplot(512, projection=ccrs.PlateCarree(central_longitude=180.0))
    ax.set_title('OLR anomalies and MJO : '+date_str[i], loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, backup[90+i,:,:], levels_olr,  transform=ccrs.PlateCarree(), cmap='RdBu_r', extend='both')
    #c = ax.contour(lon, lat, mjo2[i,:,:], levels_mjo_pos[levels_mjo_pos != 0], colors='k', linestyles="-", transform=ccrs.PlateCarree())
    c = ax.contour(lon, lat, mjo2[i,:,:], levels_mjo_neg[levels_mjo_neg != 0], colors='k', linestyles="-", transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
    
    ax = fig.add_subplot(513, projection=ccrs.PlateCarree(central_longitude=180.0))
    ax.set_title('OLR anomalies and ER : '+date_str[i], loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, backup[90+i,:,:], levels_olr,  transform=ccrs.PlateCarree(), cmap='RdBu_r', extend='both')
    c = ax.contour(lon, lat, er2[i,:,:], levels_er[levels_er != 0], colors='r', linestyles="-", transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
    
    ax = fig.add_subplot(514, projection=ccrs.PlateCarree(central_longitude=180.0))
    ax.set_title('OLR anomalies and Kelvin : '+date_str[i], loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, backup[90+i,:,:], levels_olr,  transform=ccrs.PlateCarree(), cmap='RdBu_r', extend='both')
    c = ax.contour(lon, lat, kelvin2[i,:,:], levels_kelvin[levels_kelvin != 0], colors='b', linestyles="-", transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')

    ax = fig.add_subplot(515, projection=ccrs.PlateCarree(central_longitude=180.0))
    ax.set_title('OLR anomalies and MRG : '+date_str[i], loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, backup[90+i,:,:], levels_olr,  transform=ccrs.PlateCarree(), cmap='RdBu_r', extend='both')
    c = ax.contour(lon, lat, mrg2[i,:,:], levels_mrg[levels_mrg != 0], colors='g', linestyles="-", transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', extend='max', aspect=65, shrink=1, pad=0.20, extendrect='True')
    
    figname=dir_anim+'OLR_waves_'+date_str[i]
    fig.savefig(figname+'.png')
    plt.close()

In [ ]:
def make_animation():
    nbimages=len(time_hov)
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format(dir_anim)))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    os.chdir(dir_anim)
    for f in glob.glob("*.png"):
        os.remove(f)
    os.chdir("../")
    return Image

In [ ]:
gif_filepath = dir_anim+'OLR_waves_'+date_str[0]+'-'+date_str[-1]+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
hov_levels_olr = np.arange(-100,110,10)
hov_levels_mjo_pos = np.arange(-25,-5,5)
hov_levels_mjo_neg = np.arange(0,30,5)
hov_levels_kelvin = np.arange(-25,-5,5)
hov_levels_er = np.arange(-25,-5,5)
hov_levels_mrg = np.arange(-10,-2,2)
hov_levels_low = np.arange(-10,-2,2)

fig = plt.figure(figsize=(12., 15.))
fig.suptitle('Wave filtering in OLR anomalies : '+date_str[0]+'-'+date_str[-1], fontsize=16)
ax.set_title('Black : MJO, Blue : Kelvin, Red : ER, Green : MRG, Purple : Low', loc='center')

ax=plt.subplot(1, 1, 1)
plt.gca().invert_yaxis()
ax.set_yticks(time_hov[::4])
ax.set_yticklabels(date_str[::4])
ax.set_xticklabels('')
cf = ax.contourf(lon, time_hov, data_hov, hov_levels_olr, cmap='RdBu_r', extend='both')
cb = fig.colorbar(cf, orientation='vertical', extend='max', aspect=65, shrink=1, pad=0.05, extendrect='True')
cb.set_label('W/m$^2$', size='large')
c = ax.contour(lon, time_hov, mjo_hov, hov_levels_mjo_pos[hov_levels_mjo_pos != 0], colors='k', linestyles="-")
c = ax.contour(lon, time_hov, mjo_hov, hov_levels_mjo_neg[hov_levels_mjo_neg != 0], colors='k', linestyles="--")
c = ax.contour(lon, time_hov, kelvin_hov, hov_levels_kelvin[hov_levels_kelvin != 0], colors='b', linestyles="-")
c = ax.contour(lon, time_hov, mrg_hov, hov_levels_mrg[hov_levels_mrg != 0], colors='g', linestyles="-")
c = ax.contour(lon, time_hov, er_hov, hov_levels_er[hov_levels_er != 0], colors='r', linestyles="-")
c = ax.contour(lon, time_hov, low_hov, hov_levels_low[hov_levels_low != 0], colors='purple', linestyles="-")

ax_inset = fig.add_axes([0.125, 0.03, 0.62, 0.05], projection=ccrs.PlateCarree(central_longitude=180.0))
bounds = [(lonW, lonE, -15, 15)]
ax_inset.axes.axis('tight')
ax_inset.set_yticks(np.linspace(latS, latN, 2), crs=ccrs.PlateCarree())
lat_formatter = LatitudeFormatter()
ax_inset.yaxis.set_major_formatter(lat_formatter)
ax_inset.set_xticks(np.linspace(-180, 180, 13))
ax_inset.set_xticklabels(np.linspace(-180, 180, 13))
lon_formatter = LongitudeFormatter(zero_direction_label=True)
ax_inset.xaxis.set_major_formatter(lon_formatter)
ax_inset.coastlines()
ax_inset.stock_img()
ax_inset.set_extent(*bounds, crs=ccrs.PlateCarree())

plt.show()

figname=dir_figs+'OLR_Hov_'+date_str[0]+'-'+date_str[-1]
fig.savefig(figname+'.png')

# Ressources

- Tropical Meteorology - Equatorial Waves :
https://www.researchgate.net/profile/Hanh_Nguyen40/publication/268400435_Tropical_Meteorology_Equatorial_Waves/links/54dd65eb0cf28a3d93f90f4d/Tropical-Meteorology-Equatorial-Waves.pdf
- Convectively coupled equatorial waves :
http://onlinelibrary.wiley.com/doi/10.1029/2008RG000266/abstract
- Tropical Meteorology - Equatorial Waves :
https://www.researchgate.net/profile/Hanh_Nguyen40/publication/268400435_Tropical_Meteorology_Equatorial_Waves/links/54dd65eb0cf28a3d93f90f4d/Tropical-Meteorology-Equatorial-Waves.pdf

Accès à des produits de filtrage des ondes équatoriales/MJO en temps réel :
- http://monitor.cicsnc.org/pub/mjo/v2/
- http://mikeventrice.weebly.com/tropical-waves.html
- http://mikeventrice.weebly.com/cckwmjo.html
- http://www.bom.gov.au/climate/mjo/

- http://misva.sedoo.fr/
- http://intra.cnrm.meteo.fr/moana/tropiques/hovmoller.py
- http://intra.cnrm.meteo.fr/moana/tropiques/carte.py